<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim


### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [38]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Desafío 2
- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.

### Datos
Como fuente de datos de utilizarán las novelas de "A Song of Ice and Fire" de George Martin. Las mismas fueron obtenidas de Kaggle:
https://www.kaggle.com/datasets/saurabhbadole/game-of-thrones-book-dataset/data

El dataset cotiene cinco archivos de texto, cada uno con un libro de la saga. Se comienza concatenando todos los archivos en un solo corpus:

In [39]:
import os

lista_libros = [
    "1 - A Game of Thrones.txt",
    "2 - A Clash of Kings.txt",
    "3 - A Storm of Swords.txt",
    "4 - A Feast for Crows.txt",
    "5 - A Dance with Dragons.txt"
]

# Crear un corpus en memoria
corpus = ""

for nombre in lista_libros:
    ruta_libro = os.path.join("./Dataset", nombre)
    with open(ruta_libro, "r", encoding="latin-1") as f:
        contenido = f.read()
        corpus += contenido + "\n"  # agrega un salto de línea entre libros



Se muestran las primeras líneas del texto completo, la cantidad de líneas y caracteres:

In [40]:
# Dividir el corpus en líneas
lineas = corpus.splitlines()  # crea una lista de líneas

# Mostrar las primeras 10 líneas
print("Primeras 10 líneas:")
print("\n".join(lineas[:10]))

# Cantidad de líneas
print("\nCantidad de líneas:", len(lineas))

# Cantidad total de caracteres
print("Cantidad total de caracteres:", len(corpus))


Primeras 10 líneas:
A Game Of Thrones 
Book One of A Song of Ice and Fire 
By George R. R. Martin 
PROLOGUE 
"We should start back," Gared urged as the woods began to grow dark around them. "The wildlings are 
dead." 
"Do the dead frighten you?" Ser Waymar Royce asked with just the hint of a smile. 
Gared did not rise to the bait. He was an old man, past fifty, and he had seen the lordlings come and go. 
"Dead is dead," he said. "We have no business with the dead." 
"Are they dead?" Royce asked softly. "What proof have we?" 

Cantidad de líneas: 125666
Cantidad total de caracteres: 9778338


# Preprocesamiento de texto para embeddings

Antes de entrenar embeddings, se aplican varios pasos de limpieza y tokenización del texto:

- **Convertir a minúsculas:**  
  Se unifican todas las palabras para evitar duplicados debidos a mayúsculas/minúsculas.

- **Eliminar encabezados y secciones no deseadas:**  
  Se eliminan nombres de libros, capítulos, autor, prólogo, dedicatoria, tabla de contenidos, notas de cronología y numeración de páginas.

- **Separar en oraciones:**  
  Se divide el texto usando signos de puntuación (`.`, `?`, `!`) para preservar la estructura básica de las oraciones, incluyendo diálogos y frases cortas.

- **Limpiar cada oración:**  
  Se eliminan números y puntuación interna, y se normalizan espacios en blanco para obtener texto limpio.

- **Tokenización y eliminación de *stopwords*:**  
  Cada oración se convierte en una lista de palabras y se eliminan las palabras vacías (como "the", "and", "of") que no aportan significado semántico.


**Nota:** Algunas oraciones resultan muy cortas después de este proceso, especialmente diálogos o frases muy breves. Como alternativa, se evalúa realizar la tokenización por párrafos completos.


In [41]:
import re
import string
from nltk.corpus import stopwords

def preprocesar_texto_por_oracion(text):
    """
    Recibe un texto completo y devuelve una lista de oraciones,
    cada una representada como lista de tokens limpios.
    Esta versión no depende de NLTK punkt.
    """
    # 1. Pasar a minúsculas
    text = text.lower()
    
    # 2. Eliminar encabezados no deseados
    patterns_to_remove = [
        r"a game of thrones",
        r"a clash of kings",
        r"a storm of swords",
        r"a feast for crows",
        r"a dance with dragons",
        r"book [^\n]+",          # líneas tipo "Book One of..."
        r"by george r\. r\. martin",
        r"prologue",
        r"dedication",
        r"contents",
        r"a note on chronology",
        r"a cavil on chronology",
        r"version history.*",
        r"page \d+",             # "Page XX"
    ]
    for pat in patterns_to_remove:
        text = re.sub(pat, " ", text)
    
    # 3. Separar en “oraciones” usando signos de puntuación
    oraciones = re.split(r'[.!?]+', text)
    oraciones = [s.strip() for s in oraciones if s.strip()]
    
    # 4. Tokenizar palabras y limpiar cada oración
    stop_words = set(stopwords.words("english"))
    corpus_tokens = []
    for oracion in oraciones:
        # Eliminar números y puntuación dentro de la oración
        clean_oracion = re.sub(r'\d+', ' ', oracion)
        clean_oracion = clean_oracion.translate(str.maketrans("", "", string.punctuation))
        clean_oracion = re.sub(r'\s+', ' ', clean_oracion).strip()
        
        # Tokenizar por espacios y eliminar stopwords
        tokens = [w for w in clean_oracion.split() if w not in stop_words]
        if tokens:
            corpus_tokens.append(tokens)
    
    return corpus_tokens

# Aplicar al corpus completo
corpus_tokens = preprocesar_texto_por_oracion(corpus)

print("Cantidad de oraciones tokenizadas:", len(corpus_tokens))
print("Ejemplo de primera oración tokenizada:", corpus_tokens[0])



Cantidad de oraciones tokenizadas: 155054
Ejemplo de primera oración tokenizada: ['start', 'back', 'gared', 'urged', 'woods', 'began', 'grow', 'dark', 'around']


Visualizamos los tokens de las 20 primeras oraciones:

In [42]:
for token in corpus_tokens[:20]:
    print(token)

['start', 'back', 'gared', 'urged', 'woods', 'began', 'grow', 'dark', 'around']
['wildlings', 'dead']
['dead', 'frighten']
['ser', 'waymar', 'royce', 'asked', 'hint', 'smile']
['gared', 'rise', 'bait']
['old', 'man', 'past', 'fifty', 'seen', 'lordlings', 'come', 'go']
['dead', 'dead', 'said']
['business', 'dead']
['dead']
['royce', 'asked', 'softly']
['proof']
['saw', 'gared', 'said']
['says', 'dead', 'thats', 'proof', 'enough']
['known', 'would', 'drag', 'quarrel', 'sooner', 'later']
['wished', 'later', 'rather', 'sooner']
['mother', 'told', 'dead', 'men', 'sing', 'songs', 'put']
['wet', 'nurse', 'said', 'thing', 'royce', 'replied']
['never', 'believe', 'anything', 'hear', 'womans', 'tit']
['things', 'learned', 'even', 'dead']
['voice', 'echoed', 'loud', 'twilit', 'forest']


#### Alternativa: Tokenizar por párrafos

In [43]:
import re
import string
from nltk.corpus import stopwords

def preprocesar_texto_por_parrafo(text):
    """
    Recibe un texto completo y devuelve una lista de párrafos,
    cada uno representado como lista de tokens limpios.
    """
    # 1. Pasar a minúsculas
    text = text.lower()

    # 2. Reemplazar caracteres especiales comunes
    text = text.replace("\x97", " ")  # guion largo
    text = text.replace("\x96", " ")  # guion corto
    text = text.replace("\x91", "'").replace("\x92", "'")  # comillas simples
    text = text.replace("\x93", '"').replace("\x94", '"')  # comillas dobles
    # eliminar cualquier otro caracter no ASCII
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)
    
    # 3. Eliminar encabezados no deseados
    patterns_to_remove = [
        r"a game of thrones",
        r"a clash of kings",
        r"a storm of swords",
        r"a feast for crows",
        r"a dance with dragons",
        r"book [^\n]+",
        r"by george r\. r\. martin",
        r"prologue",
        r"dedication",
        r"contents",
        r"a note on chronology",
        r"a cavil on chronology",
        r"version history.*",
        r"page \d+",
    ]
    for pat in patterns_to_remove:
        text = re.sub(pat, " ", text)
    
    # 4. Separar en párrafos usando saltos de línea
    parrafos = text.split("\n")
    parrafos = [p.strip() for p in parrafos if p.strip()]
    
    # 5. Limpiar cada párrafo y tokenizar
    stop_words = set(stopwords.words("english"))
    corpus_tokens = []
    for parrafo in parrafos:
        # Eliminar números y puntuación
        clean_parrafo = re.sub(r'\d+', ' ', parrafo)
        clean_parrafo = clean_parrafo.translate(str.maketrans("", "", string.punctuation))
        clean_parrafo = re.sub(r'\s+', ' ', clean_parrafo).strip()
        
        # Tokenizar por espacios y eliminar stopwords
        tokens = [w for w in clean_parrafo.split() if w not in stop_words]
        if tokens:
            corpus_tokens.append(tokens)
    
    return corpus_tokens

# Aplicar al corpus completo
corpus_tokens_parrafo = preprocesar_texto_por_parrafo(corpus)

print("Cantidad de párrafos tokenizados:", len(corpus_tokens_parrafo))
print("Ejemplo de primer párrafo tokenizado:", corpus_tokens_parrafo[0])

for token in corpus_tokens_parrafo[:20]:
    print(token)


Cantidad de párrafos tokenizados: 118164
Ejemplo de primer párrafo tokenizado: ['start', 'back', 'gared', 'urged', 'woods', 'began', 'grow', 'dark', 'around', 'wildlings']
['start', 'back', 'gared', 'urged', 'woods', 'began', 'grow', 'dark', 'around', 'wildlings']
['dead']
['dead', 'frighten', 'ser', 'waymar', 'royce', 'asked', 'hint', 'smile']
['gared', 'rise', 'bait', 'old', 'man', 'past', 'fifty', 'seen', 'lordlings', 'come', 'go']
['dead', 'dead', 'said', 'business', 'dead']
['dead', 'royce', 'asked', 'softly', 'proof']
['saw', 'gared', 'said', 'says', 'dead', 'thats', 'proof', 'enough']
['known', 'would', 'drag', 'quarrel', 'sooner', 'later', 'wished', 'later', 'rather']
['sooner', 'mother', 'told', 'dead', 'men', 'sing', 'songs', 'put']
['wet', 'nurse', 'said', 'thing', 'royce', 'replied', 'never', 'believe', 'anything', 'hear', 'womans']
['tit', 'things', 'learned', 'even', 'dead', 'voice', 'echoed', 'loud', 'twilit', 'forest']
['long', 'ride', 'us', 'gared', 'pointed', 'eight',

### Entrenamiento de modelos de embeddings:

Agregamos la función de callback vista en clase para informar el loss de cada época:

In [44]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

Entrenamos dos variantes del modelo, tanto con **CBOW** como con **Skipgram**:

Si bien en el ejemplo en clase entrenamos embeddings con dimensión 300, dado que observamos que las oraciones son bastante cortas, disminuimos la dimensionalidad del embedding a 100

#### Entrenamiento modelo Skipgram

In [45]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model_sg = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=100,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=5,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [46]:
w2v_model_sg.build_vocab(corpus_tokens)

# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model_sg.corpus_count)

# Cantidad de palabras encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model_sg.wv.index_to_key))

Cantidad de docs en el corpus: 155054
Cantidad de words distintas en el corpus: 12698


In [47]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model_sg.train(corpus_tokens,
                 total_examples=w2v_model_sg.corpus_count,
                 epochs=50,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 1725810.625
Loss after epoch 1: 1232323.875
Loss after epoch 2: 1223176.5
Loss after epoch 3: 1065123.0
Loss after epoch 4: 1036676.0
Loss after epoch 5: 1026184.0
Loss after epoch 6: 1004914.5
Loss after epoch 7: 945857.5
Loss after epoch 8: 922068.0
Loss after epoch 9: 918790.0
Loss after epoch 10: 906584.0
Loss after epoch 11: 954038.0
Loss after epoch 12: 948284.0
Loss after epoch 13: 894251.0
Loss after epoch 14: 885068.0
Loss after epoch 15: 930106.0
Loss after epoch 16: 842389.0
Loss after epoch 17: 852884.0
Loss after epoch 18: 819894.0
Loss after epoch 19: 860406.0
Loss after epoch 20: 806098.0
Loss after epoch 21: 810690.0
Loss after epoch 22: 805436.0
Loss after epoch 23: 801724.0
Loss after epoch 24: 808186.0
Loss after epoch 25: 846264.0
Loss after epoch 26: 843656.0
Loss after epoch 27: 788542.0
Loss after epoch 28: 797570.0
Loss after epoch 29: 821972.0
Loss after epoch 30: 790580.0
Loss after epoch 31: 830174.0
Loss after epoch 32: 788958.0
Loss afte

(44091381, 47474250)

#### Entrenamiento modelo CBOW:

In [48]:
w2v_model_cbow = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=100,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=5,      # si tienen más cores pueden cambiar este valor
                     sg=0)           # modelo 0:CBOW  1:skipgram

In [49]:
w2v_model_cbow.build_vocab(corpus_tokens)

w2v_model_cbow.train(corpus_tokens,
                 total_examples=w2v_model_cbow.corpus_count,
                 epochs=50,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 830067.125
Loss after epoch 1: 649568.75
Loss after epoch 2: 561309.875
Loss after epoch 3: 515371.75
Loss after epoch 4: 500587.75
Loss after epoch 5: 472484.0
Loss after epoch 6: 468458.75
Loss after epoch 7: 424459.0
Loss after epoch 8: 395712.5
Loss after epoch 9: 406217.5
Loss after epoch 10: 401701.0
Loss after epoch 11: 386298.5
Loss after epoch 12: 384081.5
Loss after epoch 13: 379510.5
Loss after epoch 14: 398940.5
Loss after epoch 15: 393477.0
Loss after epoch 16: 371496.0
Loss after epoch 17: 369863.0
Loss after epoch 18: 381678.0
Loss after epoch 19: 371086.0
Loss after epoch 20: 351972.0
Loss after epoch 21: 376496.0
Loss after epoch 22: 382838.0
Loss after epoch 23: 362989.0
Loss after epoch 24: 370129.0
Loss after epoch 25: 368586.0
Loss after epoch 26: 359272.0
Loss after epoch 27: 364394.0
Loss after epoch 28: 353843.0
Loss after epoch 29: 373257.0
Loss after epoch 30: 364083.0
Loss after epoch 31: 353851.0
Loss after epoch 32: 360894.0
Loss after e

(44090839, 47474250)

### Análisis de resultados

In [50]:
def mostrar_similares(model, palabra, topn=10, tipo_modelo= ""):
    """
    Muestra una tabla con los n más similares (positivos)
    y los n menos similares (negativos) a la palabra dada.
    """
    positivos = model.wv.most_similar(positive=[palabra], topn=topn)
    negativos = model.wv.most_similar(negative=[palabra], topn=topn)

    # Convertir en DataFrames
    df_pos = pd.DataFrame(positivos, columns=["Positivo", "Similitud"])
    df_neg = pd.DataFrame(negativos, columns=["Negativo", "Similitud"])

    # Combinar lado a lado
    tabla = pd.concat([df_pos, df_neg], axis=1)

    print(f"\nPalabra consultada: {palabra}\n")
    if tipo_modelo:
        print(f"\Modelo utilozado: {tipo_modelo}\n")
    print(tabla.to_string(index=False))

In [55]:
mostrar_similares(w2v_model_sg, "tyrion", 10, "Skipgram")
mostrar_similares(w2v_model_cbow, "tyrion", 10, "CBOW")


Palabra consultada: tyrion

\Modelo utilozado: Skipgram

 Positivo  Similitud  Negativo  Similitud
   cersei   0.741190   fishing   0.125149
    jaime   0.694874 attacking   0.081464
    dwarf   0.641970     nests   0.076708
crookedly   0.596002     squat   0.059596
     dany   0.594187   trained   0.059358
    kevan   0.587625   knotted   0.050335
  brienne   0.580492   favored   0.043279
    sansa   0.576284    mostly   0.037841
    varys   0.575842       at   0.033913
   alayne   0.572007    dotted   0.030109

Palabra consultada: tyrion

\Modelo utilozado: CBOW

Positivo  Similitud    Negativo  Similitud
   jaime   0.783607      called   0.419913
  cersei   0.750971       knows   0.383450
 brienne   0.730268        the   0.383371
    dany   0.692191        plus   0.375646
   dwarf   0.674594     sighted   0.373884
   sansa   0.614691 accompanied   0.370443
 catelyn   0.604138     emerged   0.369464
     ned   0.597978      driven   0.367199
    arya   0.595080        wore   0.358

In [56]:
mostrar_similares(w2v_model_sg, "jon", 10, "Skipgram")
mostrar_similares(w2v_model_cbow, "jon", 10, "CBOW")


Palabra consultada: jon

\Modelo utilozado: Skipgram

Positivo  Similitud  Negativo  Similitud
  qhorin   0.634706     favor   0.072145
    jon   0.627027   sceptre   0.070472
 catelyn   0.622429 stiffened   0.055806
    bran   0.602437    repute   0.046927
     ned   0.596018     plays   0.046914
 ygritte   0.590014     scrub   0.032574
     sam   0.585840 courtiers   0.031462
     ion   0.574884   pillars   0.029485
     fon   0.562981     bends   0.026725
   mance   0.558799    nailed   0.025407

Palabra consultada: jon

\Modelo utilozado: CBOW

Positivo  Similitud    Negativo  Similitud
    bran   0.661791     quailed   0.433029
     sam   0.635791    nobility   0.432746
 catelyn   0.615853  mistrusted   0.394515
   davos   0.614937 accompanied   0.394212
   theon   0.587482    infantry   0.385418
     ned   0.571846     planted   0.380035
     ion   0.567229      hefted   0.376840
    arya   0.566378       stews   0.375331
    dany   0.544259     gliding   0.373697
 brienne   0.

In [57]:
mostrar_similares(w2v_model_sg, "jaime", 10, "Skipgram")
mostrar_similares(w2v_model_cbow, "jaime", 10, "CBOW")


Palabra consultada: jaime

\Modelo utilozado: Skipgram

Positivo  Similitud Negativo  Similitud
  tyrion   0.694874  fishing   0.121288
  cersei   0.679603   dotted   0.075399
   kevan   0.611098    patch   0.037903
   sansa   0.600463      at   0.033800
 brienne   0.593470     toys   0.033699
 catelyn   0.572049    beams   0.033441
   loras   0.563172     pole   0.033418
   daven   0.559101   maids   0.031080
    dany   0.549354     oval   0.030966
  lancel   0.548754 longhall   0.028478

Palabra consultada: jaime

\Modelo utilozado: CBOW

Positivo  Similitud  Negativo  Similitud
  tyrion   0.783607    dotted   0.415130
  cersei   0.734387 consigned   0.388718
   kevan   0.712490 dispersed   0.383931
 brienne   0.622864    soared   0.380317
 catelyn   0.598228     woven   0.374453
   sansa   0.570295     draws   0.366896
     ned   0.551906  anchored   0.362752
   tywin   0.551361    enters   0.361224
   cleos   0.548854      the   0.361205
  lancel   0.542193  wreckage   0.350539

In [58]:
mostrar_similares(w2v_model_sg, "daenerys", 10, "Skipgram")
mostrar_similares(w2v_model_cbow, "daenerys", 10, "CBOW")


Palabra consultada: daenerys

\Modelo utilozado: Skipgram

   Positivo  Similitud    Negativo  Similitud
  stormborn   0.680650      smiths   0.152044
    unburnt   0.623447       ankle   0.076069
  targaryen   0.586436      tugged   0.076065
    viserys   0.528879    crackled   0.070229
   khaleesi   0.526044       broom   0.065783
     aegon   0.522682 cleanshaven   0.063186
sunandstars   0.519439  everywhere   0.059365
  consented   0.505674     blocked   0.059033
      queen   0.503939        luke   0.056934
     sweet   0.490403       store   0.056267

Palabra consultada: daenerys

\Modelo utilozado: CBOW

 Positivo  Similitud   Negativo  Similitud
    aerys   0.471588    blocked   0.412596
  viserys   0.467510    archery   0.393226
    queen   0.461241      pikes   0.356539
     dany   0.457213      round   0.350911
       ii   0.455923     showed   0.340260
   aegon   0.454993       hose   0.339790
   cersei   0.453849     smithy   0.338736
stormborn   0.433189     hacked   

In [59]:
mostrar_similares(w2v_model_sg, "stark", 10, "Skipgram")
mostrar_similares(w2v_model_cbow, "stark", 10, "CBOW")


Palabra consultada: stark

\Modelo utilozado: Skipgram

  Positivo  Similitud  Negativo  Similitud
   starks   0.674530  unwashed   0.066877
    starks   0.668352  overseer   0.062107
    eddard   0.649646 sharpened   0.058771
      robb   0.592073   hopping   0.058646
  eddards   0.589543    onions   0.055465
     tully   0.546825    coarse   0.051253
winterfell   0.536577    popped   0.048029
  leobalds   0.520321     manse   0.047801
     daryn   0.514446   barrels   0.043414
   cassana   0.505951   puppies   0.037317

Palabra consultada: stark

\Modelo utilozado: CBOW

   Positivo  Similitud      Negativo  Similitud
     starks   0.689264       grasses   0.449166
    starks   0.635930       firepit   0.440947
      tully   0.501797     scrabbled   0.399156
     father   0.467121          coil   0.392945
   karstark   0.456764 diamondshaped   0.392163
 winterfell   0.453886      hawthorn   0.390141
grandfather   0.450694     mushrooms   0.389273
    greyjoy   0.442022        pew

In [60]:
mostrar_similares(w2v_model_sg, "dragon", 10, "Skipgram")
mostrar_similares(w2v_model_cbow, "dragon", 10, "CBOW")


Palabra consultada: dragon

\Modelo utilozado: Skipgram

   Positivo  Similitud  Negativo  Similitud
threeheaded   0.623594 greyfaced   0.096935
    dragons   0.570713     mudge   0.091785
     drogon   0.564024   service   0.086874
   dragons   0.547834   chances   0.081100
    dynasty   0.525226      slip   0.071666
     aegon   0.508702 manatarms   0.069938
    visenya   0.507552     brune   0.067433
  stormborn   0.505543  mornings   0.065012
    viserys   0.498209 wandering   0.064896
      aegon   0.497900     rhyme   0.063901

Palabra consultada: dragon

\Modelo utilozado: CBOW

 Positivo  Similitud   Negativo  Similitud
  dragons   0.546968   terrance   0.403854
conqueror   0.436736    scowled   0.396557
 dragons   0.420863      lord   0.379978
    bitch   0.415827    stouts   0.372693
   drogon   0.411245 softspoken   0.366340
    lions   0.400173     chided   0.364933
targaryen   0.395804 complaints   0.363622
     fire   0.393629   subsided   0.355576
  meraxes   0.391

In [61]:
mostrar_similares(w2v_model_sg, "throne", 10, "Skipgram")
mostrar_similares(w2v_model_cbow, "throne", 10, "CBOW")


Palabra consultada: throne

\Modelo utilozado: Skipgram

  Positivo  Similitud  Negativo  Similitud
      iron   0.556011  messages   0.109514
     chair   0.549967      pies   0.106188
     crown   0.528103    popped   0.105961
   derives   0.515011    emrick   0.100937
rightfully   0.501939 feathered   0.083466
  rightful   0.500867    pepper   0.082216
     hinge   0.499716     dream   0.078566
birthright   0.495223     gruff   0.069395
        ii   0.488736      bill   0.065157
  proclaim   0.486143    rolled   0.060728

Palabra consultada: throne

\Modelo utilozado: CBOW

  Positivo  Similitud      Negativo  Similitud
     crown   0.507749        combed   0.368167
     chair   0.483957     stammered   0.364914
   victory   0.463250       should   0.362063
   thrones   0.454978 understanding   0.353198
      seat   0.442675       puffing   0.351534
      holt   0.427063       teasing   0.343023
birthright   0.414824         piney   0.342458
   sconces   0.414757       weaving   0

### Visualizar agrupación de vectores

In [62]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model, num_dimensions = 2 ):
     
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [ ]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model_sg)



In [65]:
MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

In [67]:
vecs_3d, labels_3d = reduce_dimensions(w2v_model_sg, 3)

In [68]:
fig = px.scatter_3d(x=vecs_3d[:MAX_WORDS,0], y=vecs_3d[:MAX_WORDS,1], z=vecs_3d[:MAX_WORDS,2],text=labels_3d[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show(renderer="colab") # esto para plotly en colab